In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
retail = pd.read_excel(r'OMITTED', sheet_name='Sheet1')

print("Shape: {}".format(retail.shape))

Shape: (6738, 9)


In [ ]:
retail.head()

# Objectives
1. Determine the number of stores per state.
2. What cities/states have the most stores?
3. Cluster stores in various cities to see how they are distributed.
4. Forecast improvements based on clusters

In [4]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

# Determine full state name 
att_state_names = []
for key, value in retail['STATE'].iteritems():
    att_state_names.append(states[value])
    
# Add full state names to dataframe
retail['STATE_NAME'] = att_state_names

In [ ]:
retail.tail()

## Number of stores per state:

In [ ]:
c = sns.countplot(x=retail['STATE_NAME'], label='Count')
c.set_xticklabels(c.get_xticklabels(), rotation=90, size=8)
plt.show()

## States with most stores (Top 10)

In [ ]:
retail.groupby('STATE_NAME').size().sort_values(ascending=False)[:10]

## Cities with most stores (Top 10):

In [ ]:
retail.groupby('CITY').size().sort_values(ascending=False)[:10]

## Plot all stores in Houston

In [9]:
import matplotlib.pyplot as plt
import matplotlib.cm
 
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize

In [10]:
Houston = retail.loc[retail['CITY'] == 'HOUSTON']
print("Number of stores in Houston: {}".format(Houston.shape[0]))

Number of stores in Houston: 53


In [11]:
# Create numpy arrays of lat/long data
h_lat = Houston['LATITUDE'].values
h_lon = Houston['LONGITUDE'].values

In [ ]:
# Obtain lat/long of Houston via --> http://boundingbox.klokantech.com/
# westlimit=-95.909744; southlimit=29.537071; eastlimit=-95.012054; northlimit=30.11035 (Houston, TX)
fig = plt.figure(figsize=(12,8), dpi=500, edgecolor='k', facecolor='w')
m = Basemap(resolution = 'l', projection = 'cyl', llcrnrlon = -95.909744, 
            llcrnrlat = 29.537071, urcrnrlon = -95.012054, urcrnrlat = 30.11035, 
            epsg = 3395)
m.scatter(h_lon, h_lat, latlon=True, s=60, edgecolor='k', linewidths=1)
m.arcgisimage(service='ESRI_StreetMap_World_2D', xpixels=2000)
plt.savefig(r'OMITTED', bbox_inches = "tight", dpi = 500)

## Apply clusterization to Houston data

In [13]:
from sklearn.cluster import DBSCAN

# Determine 'eps' parameter threshold

# Radius of earth = 3359 miles
miles_per_radian = 3359 

# define epsilon to be 3 miles, then convert to radians to be used by haversine
epsilon = 3 / miles_per_radian

# Create array of coordinates 
houston_coords = Houston[['LATITUDE', 'LONGITUDE']].values

In [14]:
# Haversine is used for spherical; Input must therefore be in radians
db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine')
db.fit(np.radians(houston_coords))

DBSCAN(algorithm='ball_tree', eps=0.0008931229532598987, leaf_size=30,
    metric='haversine', metric_params=None, min_samples=3, n_jobs=1,
    p=None)

In [15]:
# Cluster labels
cluster_labels = db.labels_

# Names of unique cluster labels
unique_labels = set(cluster_labels)

# Total number of clusters generated
number_of_cluster = len(set(cluster_labels))

print("Names of unique clusters: {}\nNumber of clusters: {}".format(unique_labels, number_of_cluster))

Names of unique clusters: {0, 1, -1}
Number of clusters: 3


In [ ]:
# Plot clusters on city map
fig = plt.figure(figsize=(12,8), dpi=500, edgecolor='k', facecolor='w')

# Create color wheel for the number of clusters
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)), alpha=0.5)

m = Basemap(resolution = 'l', projection = 'cyl', llcrnrlon = -95.909744, 
            llcrnrlat = 29.537071, urcrnrlon = -95.012054, urcrnrlat = 30.11035, 
            epsg = 3395)

for cluster_label, color  in zip(unique_labels, colors):
    # Mark the noise as red
    if cluster_label == -1:
        color = 'red'

    
    # Plot the points that match the cluster labels based on their index
    # Latitude can be positive and negative (north and south of equator) -- Y axis
    x_coords = houston_coords[cluster_labels==cluster_label][:,1] #lat
    y_coords = houston_coords[cluster_labels==cluster_label][:,0] #lon
    
    m.scatter(x=x_coords, y=y_coords, latlon=True, c=color, s=60, alpha=1.0, edgecolor='k', linewidths=1)
    
m.arcgisimage(service='ESRI_StreetMap_World_2D', xpixels=2000)

plt.savefig(r'\Houston_Cluster.png', bbox_inches = "tight", dpi = 500)